<a href="https://colab.research.google.com/github/dinuka-kasun-medis/mycolab/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-09-15 16:37:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-15 16:37:24 (47.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
#Import series of helper functions for the note book
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")


--2023-09-15 16:37:27--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.207, 209.85.147.207, 142.250.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-09-15 16:37:27 (139 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



Unzipping nlp_getting_started.zip gives the following 3 .csv files:

1. sample_submission.csv - an example of the file you'd submit to the Kaggle competition of your model's predictions.
2. train.csv - training samples of real and not real diaster Tweets.
3. test.csv - testing samples of real and not real diaster Tweets.

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")#df --> data frame
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
print(f"Total training sample: {len(train_df)}")
print(f"Total test sample: {len(test_df)}")
print(f"Total sample: {len(train_df) + len(test_df)}")

Total training sample: 7613
Total test sample: 3263
Total sample: 10876


In [9]:
#Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of sample
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 0 (not real disaster)
Text:
Still no plans? Don't worry we got you covered. Plenty of Seismic IPA and Seismic Squeeze Radler to help... http://t.co/A8nMdkd3rV

---

Target: 0 (not real disaster)
Text:
The Campaign: Will Ferrell and Zach Galifianakis commit comic mayhem in this hilarious political farce. 4* http://t.co/tQ3j2qGtZQ

---

Target: 1 (real disaster)
Text:
http://t.co/iGXRqPoTm7 Bin Laden family plane crashed after 'avoiding microlight and landi... http://t.co/3kPBU6hGt5 #PeritoEnGrafoscopia

---

Target: 0 (not real disaster)
Text:
Always look for the silver lining!  

My barn having burned to the ground
I can now see the moon.
~ Mizuta... http://t.co/Gl4McaX0ny

---

Target: 1 (real disaster)
Text:
#stormchase Violent Record Breaking EF-5 El Reno Oklahoma Tornado Nearly Runs Over ... - http://t.co/3SICroAaNz http://t.co/I27Oa0HISp

---



In [13]:
#Split data into training and validation sets
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [14]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [17]:
#first train sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [23]:
#Converting text into numbers and tokenization
from tensorflow.keras.layers import TextVectorization

#ude default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [24]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [25]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [26]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [27]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [28]:
# Create sample sentence and tokenize it
sample_sentence_2 = "ඕනෑම කෙනෙකුට වැඩිදියුණු කළහැකි නිදහස් විශ්‍‍වකෝෂ‍ය‍."
text_vectorizer([sample_sentence_2])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [29]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
tiffanyfrizzell has a crush: http://t.co/RaF732vRtt      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7566,   41,    3,  438,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [30]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [34]:
import tensorflow as tf
from tensorflow.keras import layers

tf.random.set_seed(42)

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,# set size of embedding vector
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1")

embedding

In [35]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@DannyRaynard not bad personally I'd get rid of either hazard or aguero for a better striker than berahino      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03573382,  0.04861614,  0.03765262, ...,  0.04840958,
         -0.0225094 ,  0.04789959],
        [-0.02458439, -0.04768223,  0.03796436, ...,  0.04494281,
         -0.04337345, -0.03016831],
        [-0.0120528 ,  0.02647712, -0.02841412, ..., -0.03454502,
          0.01778299,  0.00743223],
        ...,
        [ 0.01690232, -0.04182345, -0.02340516, ...,  0.00597562,
         -0.00948402,  0.04947357],
        [ 0.02031222, -0.02156004,  0.01239208, ..., -0.0372367 ,
         -0.00020241, -0.04340919],
        [-0.04375411,  0.01341991,  0.03812731, ...,  0.00620025,
          0.04009812, -0.02144527]]], dtype=float32)>

In [36]:
# Check out a single token's embedding
sample_embed[0][0]


<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03573382,  0.04861614,  0.03765262, -0.01561087,  0.00477878,
        0.03515523, -0.04754791, -0.02621902,  0.00733496,  0.03410535,
       -0.0104183 , -0.01706899, -0.00669105, -0.0002862 ,  0.00967885,
       -0.00214094, -0.03186379,  0.03981045,  0.03363388,  0.02857434,
        0.00239088,  0.04354818, -0.04850863,  0.04855717,  0.04340794,
       -0.03709329,  0.03133066,  0.02900107, -0.03166842,  0.03935449,
       -0.01902069,  0.02985391,  0.0478864 ,  0.03716514, -0.04667084,
        0.04587563,  0.02416576, -0.03447254,  0.00096587, -0.02275808,
        0.00728745, -0.00137473,  0.0133529 , -0.02911737, -0.03461802,
       -0.04548857, -0.02029994, -0.02606593,  0.01174117, -0.02663145,
        0.0188226 ,  0.0278796 ,  0.02904296, -0.03039943, -0.03378381,
       -0.00727464, -0.0227273 , -0.02049236,  0.04175056,  0.02626527,
       -0.00900266,  0.01662865,  0.04731866,  0.03039673,  0.01438682,
        0.039402